In [25]:
from unittest.mock import inplace

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split,StratifiedShuffleSplit
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder,OneHotEncoder,StandardScaler
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.preprocessing import  MinMaxScaler
from sklearn.pipeline import Pipeline

In [26]:

df_original = pd.read_csv(
    'https://raw.githubusercontent.com/anvarnarz/praktikum_datasets/main/housing_data_08-02-2021.csv')
df_original.head().sort_values(by='price', ascending=True)

,location,district,rooms,size,level,max_levels,price
2,"город Ташкент, Чиланзарский район, Чиланзар 2-...",Чиланзарский,2,42,4,4,37000
3,"город Ташкент, Чиланзарский район, Чиланзар 9-...",Чиланзарский,3,65,1,4,49500
0,"город Ташкент, Юнусабадский район, Юнусабад 8-...",Юнусабадский,3,57,4,4,52000
4,"город Ташкент, Чиланзарский район, площадь Актепа",Чиланзарский,3,70,3,5,55000
1,"город Ташкент, Яккасарайский район, 1-й тупик ...",Яккасарайский,2,52,4,5,56000


In [27]:
stratified_split=StratifiedShuffleSplit(n_splits=1,test_size=0.2,random_state=13)
for train ,test in stratified_split.split(df_original,df_original['district']):
    train_set=df_original.loc[train]
    test_set=df_original.loc[test]

train_set1=train_set.copy()

train_set_y=train_set.copy()

test_set1=test_set.copy()

In [28]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin

class changed(BaseEstimator, TransformerMixin):
    def __init__(self, y_label=False):
        self.y_label = y_label

    def fit(self, df, y=None):
        return self

    def transform(self, df):
        df['size'] = pd.to_numeric(df['size'], errors='coerce')
        df['streat'] = df['location'].str.split(',').map(lambda x: x[-1])
        df['size'] = df.groupby(['district', 'rooms', 'level', 'max_levels'])['size'].transform(lambda x: x.fillna(x.median()))
        df['sizemd'] = df.groupby(['rooms', 'level', 'max_levels', 'district'])['size'].transform('median')
        df['location'] = pd.factorize(df['location'])[0]
        df['district'] = pd.factorize(df['district'])[0]
        df['streat'] = pd.factorize(df['streat'])[0]
        df.loc[df['size'] > 500, 'size'] = df['sizemd']
        df.loc[df['size'].isna(), 'size'] = df['sizemd']
        if self.y_label:
            df['price'] = pd.to_numeric(df['price'], errors='coerce')
            df['price'] = df.groupby(['district', 'size', 'rooms', 'level'])['price'].transform(lambda x: x.fillna(x.median()))
            df['pricemd'] = df.groupby(['rooms', 'level'])['price'].transform('median')
            df.loc[df['price'] > 1000000, 'price'] = df['pricemd']
            df.loc[df['price'].isna(), 'price'] = df['pricemd']
            df.fillna(df.median(),inplace=True)
            return df.pop('price')
        else:
            df.drop(columns=['price'],inplace=True)
            df.fillna(df.median(),inplace=True)
            return df

In [29]:
ful_pipeline_y=Pipeline([
    ('change', changed(y_label=True))
])


In [30]:
y=ful_pipeline_y.fit_transform(train_set_y)

C:\Users\USER\.virtualenvs\PyCharmLearningProject\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1241: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\USER\.virtualenvs\PyCharmLearningProject\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1241: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\USER\.virtualenvs\PyCharmLearningProject\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1241: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\USER\.virtualenvs\PyCharmLearningProject\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1241: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\USER\.virtualenvs\PyCharmLearningProject\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1241: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Us

In [31]:
ful_pipeline=Pipeline([
    ('change', changed()),
    ('standarted',StandardScaler())
])
x=ful_pipeline.fit_transform(train_set)

In [32]:
from sklearn.linear_model import LinearRegression

LR_model = LinearRegression()

In [33]:
LR_model.fit(x,y)

LinearRegression()

In [34]:
test1=train_set1.sample(15)
test2=test1.copy()

In [35]:
test1=ful_pipeline.fit_transform(test1)

In [36]:
pr=LR_model.predict(test1)

In [37]:
pd.DataFrame({'pr':pr,'or':test2['price']})

,pr,or
3410,39736.452628,35000
4850,61268.030129,49000
691,66553.104422,52500
3856,61335.596880,51000
5981,44371.648127,34500
4463,65567.898350,47000
1862,79123.323980,88000
7131,29877.978002,23000
5288,52755.695962,49500
1292,74849.246756,55000


In [38]:
ts1=ful_pipeline.fit_transform(test_set)

In [39]:
tsy=ful_pipeline_y.fit_transform(test_set1)

C:\Users\USER\.virtualenvs\PyCharmLearningProject\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1241: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\USER\.virtualenvs\PyCharmLearningProject\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1241: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\USER\.virtualenvs\PyCharmLearningProject\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1241: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\USER\.virtualenvs\PyCharmLearningProject\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1241: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\USER\.virtualenvs\PyCharmLearningProject\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1241: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Us

In [40]:
pree=LR_model.predict(ts1)

In [41]:
pd.DataFrame({'pr':pree,'or':tsy})

,pr,or
221,71803.271797,81772.0
294,47326.361130,38000.0
3027,73813.161153,52000.0
6486,72856.061024,52500.0
4095,58463.370997,52000.0
...,...,...
3119,69366.933071,74000.0
5887,62242.558704,50000.0
1284,68345.902180,50670.0
4620,47846.447408,35000.0


In [42]:
from sklearn.metrics import mean_absolute_error
mea=mean_absolute_error(pree,tsy)

In [43]:
mea

np.float64(20266.098153100847)

In [44]:
20264/1500

13.509333333333334